In [ ]:
# ============================================================
# Cross-Asset HYG Factor Model 
# ============================================================


# Import Libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit

import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")



# ------------------------------------------------------------
# Utility Functions
# ------------------------------------------------------------

def log_return(series):
    return np.log(series).diff()

def safe_diff(series):
    return series.diff()

# ------------------------------------------------------------
# Data Loading Helper
# ------------------------------------------------------------

def load_excel_series(path, sheet, column=None):
    df = pd.read_excel(path, sheet_name=sheet)
    df = df.set_index("Date").sort_index()
    if column:
        return df[column]
    return df

# ------------------------------------------------------------
# Load Data
# ------------------------------------------------------------

portfolio = load_excel_series("Workshop Data.xlsx", "Portfolio")
hyg_vol   = load_excel_series("Workshop Data.xlsx", "Adj HYG", "Volume")

hy_index  = load_excel_series("Indexes and Spreads Data 01.09.xlsx", "HY Index")
ig_index  = load_excel_series("Indexes and Spreads Data 01.09.xlsx", "IG Index")
ust10     = load_excel_series("Indexes and Spreads Data 01.09.xlsx", "10yUST Yields")
hyg_yas   = load_excel_series("Indexes and Spreads Data 01.09.xlsx", "HYG")

# ------------------------------------------------------------
# HYG Total Return 
# ------------------------------------------------------------

if "TotalReturnsHYG" not in portfolio.columns:
    portfolio["HYGCumDiv"] = portfolio["HYG Dividends"][::-1].cumsum()[::-1]
    portfolio["TotalReturnsHYG"] = (
        portfolio["HYG Position"] + portfolio["HYGCumDiv"]
    )

# ------------------------------------------------------------
# Dependent Variable 
# ------------------------------------------------------------

hyg_return = portfolio["TotalReturnsHYG"].pct_change()

# ------------------------------------------------------------
# Equity Proxy (Risk-On)
# ------------------------------------------------------------

equity_px = portfolio["SPY Position"] / -10
equity_return = log_return(equity_px)

# ------------------------------------------------------------
# Duration
# ------------------------------------------------------------

hyg_duration = hyg_yas["YAS_MOD_DUR"]

# ------------------------------------------------------------
# Factor Construction (STATIONARY & ORTHOGONAL)
# ------------------------------------------------------------

# Credit spread shocks
d_hy_oas = safe_diff(hy_index["OAS_SOVEREIGN_CURVE"])
d_ig_oas = safe_diff(ig_index["OAS_SOVEREIGN_CURVE"])

# Rates shock
d_ust10 = safe_diff(ust10["PX_LAST"])

# Liquidity shock
liquidity = log_return(hyg_vol)

# Orthogonal credit factors
credit_level = 0.5 * d_hy_oas + 0.5 * d_ig_oas
credit_rotation = d_hy_oas - d_ig_oas

# Duration-adjusted rate factor
rate_change = - hyg_duration * d_ust10

# Assemble factor matrix
factors = pd.DataFrame({
    "Rate": rate_change,
    "Credit_Level": credit_level,
    "Credit_Rotation": credit_rotation,
    "Equity": equity_return,
    "Liquidity": liquidity
})

# Drop missing values and align Y
factors = factors.dropna()
hyg_return = hyg_return.loc[factors.index]

# ------------------------------------------------------------
# Regression
# ------------------------------------------------------------

X = sm.add_constant(factors)
Y = hyg_return

model = sm.OLS(Y, X).fit()

print("\n================ Regression Summary ================")
print(model.summary())

# ------------------------------------------------------------
# Diagnostics
# ------------------------------------------------------------

condition_number = np.linalg.cond(X)

print("\n================ Diagnostics ================")
print(f"Condition Number : {condition_number:.2f}")
print(f"R²              : {model.rsquared:.3f}")
print(f"Adj R²          : {model.rsquared_adj:.3f}")
print("Durbin-Watson   :", sm.stats.stattools.durbin_watson(model.resid))

# ------------------------------------------------------------
# Factor Attribution
# ------------------------------------------------------------

betas = model.params.drop("const")
attribution = factors.mul(betas, axis=1)
attribution["Total"] = attribution.sum(axis=1)

print("\nLatest Factor Attribution:")
display(attribution.tail())

# ------------------------------------------------------------
# Factor Correlation Check
# ------------------------------------------------------------

print("\nFactor Correlations:")
display(factors.corr())



================ Regression Summary ================
                            OLS Regression Results                            
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.565
Method:                 Least Squares   F-statistic:                     323.7
Date:                Wed, 14 Jan 2026   Prob (F-statistic):          8.24e-222
Time:                        17:11:53   Log-Likelihood:                 5585.7
No. Observations:                1244   AIC:                        -1.116e+04
Df Residuals:                    1238   BIC:                        -1.113e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

,Rate,Credit_Level,Credit_Rotation,Equity,Liquidity,Total
Date,,,,,,
2025-12-24,0.000847,0.000517,-0.000007,-9.611853e-07,0.000527,0.001883
2025-12-26,0.000167,-0.001237,0.000613,-3.483918e-06,-0.000023,-0.000483
2025-12-29,0.000502,-0.000352,-0.000092,-3.112080e-06,-0.000441,-0.000386
2025-12-30,-0.000336,0.001466,-0.000510,1.694518e-06,0.000056,0.000678
2025-12-31,-0.001280,0.000164,-0.000235,2.283719e-07,0.000090,-0.001261



Factor Correlations:


,Rate,Credit_Level,Credit_Rotation,Equity,Liquidity
Rate,1.000000,0.259188,0.264701,0.021210,0.031510
Credit_Level,0.259188,1.000000,0.957047,-0.008872,0.133045
Credit_Rotation,0.264701,0.957047,1.000000,-0.004137,0.123394
Equity,0.021210,-0.008872,-0.004137,1.000000,-0.046599
Liquidity,0.031510,0.133045,0.123394,-0.046599,1.000000


In [7]:
# ============================================================
# Cross-Asset HYG Factor Model — ML Extensions
# ============================================================

# ------------------------------------------------------------
# Train / Test Split 
# ------------------------------------------------------------

split_date = factors.index[int(len(factors) * 0.8)]

X_train = factors.loc[:split_date]
X_test  = factors.loc[split_date:]

y_train = hyg_return.loc[X_train.index]
y_test  = hyg_return.loc[X_test.index]

# ------------------------------------------------------------
# Evaluation Helper
# ------------------------------------------------------------

def evaluate_model(name, model, X_test, y_test, y_pred):
    return {
        "Model": name,
        "R²": r2_score(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "MAE": mean_absolute_error(y_test, y_pred)
    }

results = []

# ------------------------------------------------------------
# Ridge Regression
# ------------------------------------------------------------

ridge = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=10.0))
])

ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)

results.append(
    evaluate_model("Ridge", ridge, X_test, y_test, ridge_pred)
)

# ------------------------------------------------------------
# LASSO Regression
# ------------------------------------------------------------

lasso = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Lasso(alpha=0.001, max_iter=10000))
])

lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)

results.append(
    evaluate_model("LASSO", lasso, X_test, y_test, lasso_pred)
)

# ------------------------------------------------------------
# Random Forest
# ------------------------------------------------------------

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=5,
    min_samples_leaf=20,
    random_state=42
)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

results.append(
    evaluate_model("Random Forest", rf, X_test, y_test, rf_pred)
)

# ------------------------------------------------------------
# XGBoost
# ------------------------------------------------------------

xgb_model = xgb.XGBRegressor(
    n_estimators=400,
    max_depth=4,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

results.append(
    evaluate_model("XGBoost", xgb_model, X_test, y_test, xgb_pred)
)

# ------------------------------------------------------------
# LightGBM
# ------------------------------------------------------------

lgb_model = lgb.LGBMRegressor(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

lgb_model.fit(X_train, y_train)
lgb_pred = lgb_model.predict(X_test)

results.append(
    evaluate_model("LightGBM", lgb_model, X_test, y_test, lgb_pred)
)

# ------------------------------------------------------------
# Results Table
# ------------------------------------------------------------

results_df = pd.DataFrame(results).set_index("Model")

print("\n================ Model Evaluation =================")
display(results_df.sort_values("R²", ascending=False))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 996, number of used features: 5
[LightGBM] [Info] Start training from score -0.000243
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

,R²,RMSE,MAE
Model,,,
Ridge,0.598568,0.002220,0.001356
XGBoost,0.568963,0.002300,0.001439
LightGBM,0.563922,0.002314,0.001464
Random Forest,0.474905,0.002539,0.001531
LASSO,0.433554,0.002637,0.001631


In [5]:
# ============================================================
# Cross-Asset PCA for HYG Factor Model
# ============================================================

factors = pd.DataFrame({
    "Rate": rate_change,
    "Credit_Level": credit_level,
    "Credit_Rotation": credit_rotation,
    "Equity": equity_return,
    "Liquidity": liquidity
}).dropna()

# HYG returns aligned with factors
hyg_returns = portfolio["TotalReturnsHYG"].pct_change().loc[factors.index]

# ------------------------------------------------------------
# 2. STANDARDIZE FACTORS 
# ------------------------------------------------------------
scaler = StandardScaler()
Z = scaler.fit_transform(factors)

Z = pd.DataFrame(Z, index=factors.index, columns=factors.columns)

# ------------------------------------------------------------
# 3. RUN PCA
# ------------------------------------------------------------
pca = PCA()
PCs = pca.fit_transform(Z)

pc_df = pd.DataFrame(
    PCs,
    index=Z.index,
    columns=[f"PC{i+1}" for i in range(PCs.shape[1])]
)

# ------------------------------------------------------------
# 4. VARIANCE EXPLAINED
# ------------------------------------------------------------
explained_var = pd.Series(
    pca.explained_variance_ratio_,
    index=pc_df.columns,
    name="ExplainedVariance"
)

cumulative_var = explained_var.cumsum()

print("\n=== PCA Variance Explained ===")
print(pd.concat([explained_var, cumulative_var.rename("Cumulative")], axis=1))

# ------------------------------------------------------------
# 5. PCA LOADINGS (ECONOMIC INTERPRETATION)
# ------------------------------------------------------------
loadings = pd.DataFrame(
    pca.components_.T,
    index=factors.columns,
    columns=pc_df.columns
)

print("\n=== PCA Loadings ===")
print(loadings)

# ------------------------------------------------------------
# 6. REGRESSION USING TOP PCs
# ------------------------------------------------------------

TOP_PCS = ["PC1", "PC2", "PC3"]

X_pc = sm.add_constant(pc_df[TOP_PCS])
Y = hyg_returns.loc[X_pc.index]

pc_model = sm.OLS(
    Y,
    X_pc
).fit(
    cov_type="HAC",
    cov_kwds={"maxlags": 5}
)

print("\n=== PCA-Based Regression ===")
print(pc_model.summary())

# ------------------------------------------------------------
# 7. FACTOR ATTRIBUTION USING PCA
# ------------------------------------------------------------
latest_date = X_pc.index[-1]

latest_exposure = X_pc.loc[latest_date]
latest_contribution = latest_exposure * pc_model.params

pca_attribution = pd.DataFrame({
    "Exposure": latest_exposure,
    "Beta": pc_model.params,
    "Contribution": latest_contribution
})

print("\n=== Latest PCA Factor Attribution ===")
print(pca_attribution)

# ------------------------------------------------------------
# 8. MAP PCS BACK TO ECONOMIC FACTORS
# ------------------------------------------------------------

economic_mapping = loadings[TOP_PCS]

print("\n=== Economic Interpretation of PCs ===")
print(economic_mapping)

# ------------------------------------------------------------
# 9. SANITY CHECKS
# ------------------------------------------------------------
print("\n=== Diagnostics ===")
print(f"R²            : {pc_model.rsquared:.3f}")
print(f"Adj R²        : {pc_model.rsquared_adj:.3f}")
print(f"Durbin-Watson : {sm.stats.stattools.durbin_watson(pc_model.resid):.3f}")




=== PCA Variance Explained ===
     ExplainedVariance  Cumulative
PC1           0.422691    0.422691
PC2           0.208142    0.630833
PC3           0.187652    0.818486
PC4           0.172939    0.991425
PC5           0.008575    1.000000

=== PCA Loadings ===
                      PC1       PC2       PC3       PC4       PC5
Rate             0.315496  0.229853 -0.153049  0.907848 -0.004294
Credit_Level     0.661136  0.015942 -0.050138 -0.245591 -0.706977
Credit_Rotation  0.661203  0.026540 -0.056059 -0.242607  0.707181
Equity          -0.008476  0.779541  0.619793 -0.089949 -0.003057
Liquidity        0.161556 -0.581829  0.766013  0.220338  0.007095

=== PCA-Based Regression ===
                            OLS Regression Results                            
Dep. Variable:        TotalReturnsHYG   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     